In [5]:
import pandas as pd
import re
import spacy
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
df = pd.read_csv(r"../data/cases.csv")

### Clasify Cases

In [9]:
criminal_cases = df["text"].str.contains(r"CRIMINAL")
civil_cases = df["text"].str.contains(r"CIVIL")
df["civil"] = civil_cases & ~(civil_cases & criminal_cases)
df["criminal"] = criminal_cases & ~(civil_cases & criminal_cases)
# there are cases possible which are civil as well as criminal which can be handled using civil_cases & criminal_cases count=69
df["not-defined"] = ~(civil_cases | criminal_cases)

In [4]:
# 4 types possible:
# civil
# criminal
# civil and criminal
# not defined

### Clean Text

In [8]:
df.shape

(25100, 6)

In [5]:
# remove empty and non english rows
# eng = df["text"].str.contains(re.compile(r"a"))
# df = df[eng]

In [6]:
df.head()

,file-name,text,tag
0,37482.txt,CRL.A. No..2325of 2009 @ SLP(CRL.) No. 7359 of...,NaN
1,35177_2015_Judgement_08-Dec-2017.txt,...,NaN
2,40216.txt,...,NaN
3,30829.txt,http://JUDIS.NIC.IN SUPREME COU...,NaN
4,14590.txt,http://JUDIS.NIC.IN SUPREME C...,NaN


In [9]:
def clean_text(row):
    text = re.sub(r"\.", "", row["text"])
    return re.sub(r"\s+", " ", text).replace(".", "").strip()

In [10]:
# %time df['text']=df.parallel_apply(clean_text,axis=1)

CPU times: user 2.61 s, sys: 1.77 s, total: 4.38 s
Wall time: 14.9 s


In [11]:
df.head()

,file-name,text,tag,civil,criminal,not-defined
0,37482.txt,CRLA No2325of 2009 @ SLP(CRL) No 7359 of 2009 ...,NaN,False,True,False
1,35177_2015_Judgement_08-Dec-2017.txt,NON-REPORTABLE IN THE SUPREME COURT OF INDIA C...,NaN,True,False,False
2,40216.txt,NON-REPORTABLE IN THE SUPREME COURT OF INDIA C...,NaN,True,False,False
3,30829.txt,http://JUDISNICIN SUPREME COURT OF INDIA Page ...,NaN,True,False,False
4,14590.txt,http://JUDISNICIN SUPREME COURT OF INDIA Page ...,NaN,False,False,True


In [10]:
df.to_csv(r"../data/cases-tagging-raw.csv", index=False)